# Notebook 6 : Clustering

Notebook préparé par [Chloé-Agathe Azencott](http://cazencott.info) avec l'aide d'[Arthur Imbert](https://github.com/Henley13).

Dans ce notebook il s'agit d'explorer plusieurs techniques de clustering.

In [ ]:
# charger numpy as np, matplotlib as plt
%pylab inline 

In [ ]:
plt.rc('font', **{'size': 12}) # règle la taille de police globalement pour les plots (en pt)

In [ ]:
import pandas as pd

## 1. Cercles imbriqués

Générons un jeu de données en deux dimensions formé de deux cercles imbriqués :

In [ ]:
from sklearn import datasets

In [ ]:
# nombre de points
n_samples = 1500

# set random seed
np.random.seed(37)

circles_X, circles_labels = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)

Visualisons ces données :

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=circles_labels)

Supposons maintenant ne pas disposer des étiquettes. Quels algorithmes de clustering permettent de trouver deux clusters, correspondant chacun à un des cercles ?

### Algorithme des k-moyennes

L'objectif de l'algorithme k-means est retrouver $K$ clusters (et leur centroïde $\mu_k$) de manière à **minimiser la variance intra-cluster** :

\begin{align}
V = \sum_{k = 1}^{K} \sum_{x \in C_k} \frac{1}{|C_k|} (\|x - \mu_k\|^2)
\end{align}

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
from sklearn import cluster

In [ ]:
# initialisation d'un k-means avec k=2
kmeans = cluster.KMeans(n_clusters=2)

# application aux données 
kmeans.fit(circles_X)

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
kmeans.labels_

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=kmeans.labels_)
plt.title("Clustering K-means (K=2)")

#### Trouver K avec le coefficient de silhouette

Le coefficient (ou score) de silhouette permet de **comparer les distances moyennes intra- et inter-cluster** :

\begin{align}
\text{score} = \frac{b - a}{\max(a, b)}
\end{align}

avec $a$ la distance moyenne intra-cluster et $b$ la distance d'un point au cluster étranger le plus proche. Le score se calcule par observation (avec une valeur entre -1 et 1) puis la moyenne de ce score permet d'évaluer le clustering du nuage de point dans son ensemble.

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score

In [ ]:
from sklearn import metrics

In [ ]:
print("Coefficient de silhouette pour le k-means (k=2) : %.2f" % metrics.silhouette_score(circles_X, kmeans.labels_))

In [ ]:
silhouettes = []
k_values = range(2, 9)
for kval in k_values:
    kmeans_k = cluster.KMeans(n_clusters=kval)
    kmeans_k.fit(circles_X)
    silhouettes.append(metrics.silhouette_score(circles_X, kmeans_k.labels_))

In [ ]:
plt.plot(k_values, silhouettes)

plt.xlabel("K")
plt.ylabel("silhouette")

print("Coefficient de silhouette du KMeans en fonction de K")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
best_K = k_values[silhouettes.index(best_silhouette)]
print("K correspondant : %.2f" % best_K)

In [ ]:
kmeans_k = cluster.KMeans(n_clusters=best_K)
kmeans_k.fit(circles_X)

fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=kmeans_k.labels_)
plt.title("Clustering K-means (K=%d)" % best_K)

### DBSCAN (Clustering par densité)

L'algorithme DBSCAN (Density-Based Spatial Clustering of Applications with Noise) fonctionne en deux temps :
- Toutes les observations suffisamment proches sont connectées entre elles.
- Les observations avec un nombre minimal de voisins connectés sont considérées comme des *core samples*, à partir desquelles les clusters sont étendues. **Toutes les observations suffisamment proche d'un *core sample* appartiennent au même cluster que celui-ci**. 

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

In [ ]:
# initialisation d'un clustering DBSCAN
dbscan = cluster.DBSCAN(eps=0.2, min_samples=2)

# application aux données 
dbscan.fit(circles_X)

In [ ]:
np.unique(dbscan.labels_)

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=dbscan.labels_)
plt.title("Clustering DBSCAN (eps=0.2)")

#### Rôle du paramètre de taille de voisinage (`eps`)

Si `eps` est trop petit :

In [ ]:
# initialisation d'un clustering DBSCAN
dbscan_005 = cluster.DBSCAN(eps=0.05, min_samples=2)

# application aux données 
dbscan_005.fit(circles_X)

In [ ]:
np.unique(dbscan_005.labels_)

L'attribut `.labels_` contient, pour chaque observation, le numéro du cluster auquel cette observation est assignée.

In [ ]:
fig = plt.figure(figsize=(5, 5))

outliers = np.where(dbscan_005.labels_ == -1)[0]
plt.scatter(circles_X[outliers, 0], circles_X[outliers, 1], marker='*', color='red')

non_outliers = np.where(dbscan_005.labels_ != -1)[0]
plt.scatter(circles_X[non_outliers, 0], circles_X[non_outliers, 1], c=dbscan_005.labels_[non_outliers])
plt.title("Clustering DBSCAN (eps=0.05)")

Si `eps` est trop grand :

In [ ]:
# initialisation d'un clustering DBSCAN
dbscan_2 = cluster.DBSCAN(eps=2., min_samples=2)

# application aux données 
dbscan_2.fit(circles_X)

In [ ]:
np.unique(dbscan_2.labels_)

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.scatter(circles_X[:, 0], circles_X[:, 1], c=dbscan_2.labels_)
plt.title("Clustering DBSCAN (eps=2.)")

#### Trouver eps avec le coefficient de silhouette

In [ ]:
print("Coefficient de silhouette pour DBSCAN (eps=0.2) : %.2f" % metrics.silhouette_score(circles_X, dbscan.labels_))

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = cluster.DBSCAN(eps=eps, min_samples=2)
    dbscan_eps.fit(circles_X)
    if len(unique(dbscan_eps.labels_)) > 1: # nécessaire pour calculer le coeff de silhouette
        silhouettes.append(metrics.silhouette_score(circles_X, dbscan_eps.labels_))
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (échelle log)")
plt.ylabel("silhouette")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
print("Eps correspondant : %.2f" % eps_values[silhouettes.index(best_silhouette)])

### Index de Rand ajusté

L'index de Rand ajusté permet de **comparer un résultat de clustering avec des étiquettes**. Pour chaque paire d'observations nous regardons si elles se situent dans le même cluster ou non, dans le clustering prédit et réel. L'index prend des valeurs entre 0 (clustering aléatoire) et 1 (clustering parfait).

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html

__Question :__ Pourquoi ne pas utiliser une métrique d'évaluation de modèle de classification ici ?

In [ ]:
print("Index de Rand ajusté du K-means (K=2) : %.2f" % metrics.adjusted_rand_score(circles_labels, kmeans.labels_))

In [ ]:
print("Index de Rand ajusté de dbscan (eps=0.2) : %.2f" % metrics.adjusted_rand_score(circles_labels, dbscan.labels_))

## 2. Manchots

On reprend ici les données utilisées dans le notebook 4

In [ ]:
palmerpenguins = pd.read_csv("data/penguins_data.csv")

__Alternativement :__ Si vous avez besoin de télécharger le fichier (par exemple sur colab) :

In [ ]:
!wget https://raw.githubusercontent.com/chagaz/cp-ia-intro-ml-2022/main/4-SVM/data/penguins_data.csv

palmerpenguins = pd.read_csv("penguins_data.csv")

In [ ]:
palmerpenguins = palmerpenguins[palmerpenguins['bill_depth_mm'].notna()]
palmerpenguins = palmerpenguins.reset_index()

In [ ]:
penguins_X = np.array(palmerpenguins[["bill_length_mm", "body_mass_g"]])

In [ ]:
from sklearn import preprocessing

In [ ]:
# standardisation (centrer-réduire)
penguins_X = preprocessing.StandardScaler().fit_transform(penguins_X)

In [ ]:
species_names, species_int = np.unique(palmerpenguins.species, return_inverse=True)
penguins_labels = species_int

In [ ]:
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels)
plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

### KMeans

In [ ]:
# initialisation d'un k-means avec k=3
kmeans = cluster.KMeans(n_clusters=3)

# application aux données 
kmeans.fit(penguins_X)

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=kmeans.labels_, marker='*')


plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

In [ ]:
print("Coefficient de silhouette pour le k-means (k=3) : %.2f" % metrics.silhouette_score(penguins_X, kmeans.labels_))

In [ ]:
print("Index de Rand ajusté du K-means (K=3) : %.2f" % metrics.adjusted_rand_score(penguins_labels, kmeans.labels_))

### DBSCAN

In [ ]:
eps_values = np.logspace(-3, 1, 40)
silhouettes = []

for eps in eps_values:
    dbscan_eps = cluster.DBSCAN(eps=eps, min_samples=2)
    dbscan_eps.fit(penguins_X)
    if len(unique(dbscan_eps.labels_)) > 1: # nécessaire pour calculer le coeff de silhouette
        silhouettes.append(metrics.silhouette_score(penguins_X, dbscan_eps.labels_))
    else:
        silhouettes.append(-1)

In [ ]:
plt.plot(eps_values, silhouettes)
plt.xscale("log")
plt.xlabel("eps (échelle log)")
plt.ylabel("silhouette")

In [ ]:
best_silhouette = np.max(silhouettes)
print("Coefficient de silhouette optimal : %.2f" % best_silhouette)
best_eps = eps_values[silhouettes.index(best_silhouette)]
print("Eps correspondant : %.2f" % best_eps)

In [ ]:
dbscan_opt = cluster.DBSCAN(eps=best_eps, min_samples=2)
dbscan_opt.fit(penguins_X)

In [ ]:
np.unique(dbscan_opt.labels_)

In [ ]:
print("Index de Rand ajusté de DBSCAN : %.2f" % metrics.adjusted_rand_score(penguins_labels, dbscan_opt.labels_))

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=dbscan_opt.labels_, marker='*')


plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

### Modèle de mélange gaussien 

Le modèle de mélange de gaussiennes cherche à **optimiser les paramètres d'un nombre fini de gaussiennes** aux données. 

Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html

In [ ]:
from sklearn import mixture

In [ ]:
# initialisation d'un k-means avec k=3
gmm = mixture.GaussianMixture(n_components=3)

# application aux données 
gmm.fit(penguins_X)

# prédiction des clusters
gmm_labels = gmm.predict(penguins_X)

In [ ]:
#plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=penguins_labels, marker='o')
plt.scatter(penguins_X[:, 0], penguins_X[:, 1], c=gmm_labels, marker='*')


plt.xlabel("bill_length_mm (centrée-réduite)")
plt.ylabel("body_mass_g (centrée-réduite)")

In [ ]:
print("Coefficient de silhouette pour le GMM (k=3) : %.2f" % metrics.silhouette_score(penguins_X, gmm_labels))

In [ ]:
print("Index de Rand ajusté du GMM (K=3) : %.2f" % metrics.adjusted_rand_score(penguins_labels, gmm_labels))